In [3]:
import random

COMB_ID = 0
ASCII_OFFSET = 399
EMPTY_PLACEHOLDER = "_"

def create_combination(name, expected_time):
    global COMB_ID
    COMB_ID += 1
    return {
        "name": name,
        "time": expected_time,
        "id": COMB_ID
    }

def create_workout(num_rounds, round_time, rest_time, comb_gap, combinations):
    workout = []
    for i in range(num_rounds):
        schedule = [EMPTY_PLACEHOLDER] * round_time
        second = comb_gap
        while second < round_time:
            combination = random.choice(combinations)
            for i in range(second, min(second+combination["time"], round_time)):
                schedule[i] = chr(ASCII_OFFSET+combination["id"])
            second += combination["time"] + comb_gap
        schedule[::len(schedule)-1] = [chr(ASCII_OFFSET)]*2
        workout = [*workout, *schedule] + [EMPTY_PLACEHOLDER]*rest_time
    return workout[:-rest_time]

In [4]:
COMBINATIONS = [
    create_combination("up and around", 3),
    create_combination("slip and go", 4),
    create_combination("hook body", 4),
    create_combination("angles", 2),
    create_combination("double up", 4)
]
NUM_ROUNDS = 5
ROUND_LENGTH = 60
REST_LENGTH = 30
COMB_GAP = 2

workout = create_workout(NUM_ROUNDS, ROUND_LENGTH, REST_LENGTH, COMB_GAP, COMBINATIONS)

In [6]:
k = "".join(workout).split("_"*30)
k

['Ə_ƒƒƒƒ__ƔƔƔƔ__ƑƑƑƑ__ƔƔƔƔ__ƓƓ__ƑƑƑƑ__ƓƓ__ƓƓ__ƒƒƒƒ__ƐƐƐ__ƔƔƔƔƏ',
 'Ə_ƔƔƔƔ__ƔƔƔƔ__ƐƐƐ__ƓƓ__ƓƓ__ƑƑƑƑ__ƓƓ__ƑƑƑƑ__ƐƐƐ__ƐƐƐ__ƐƐƐ__ƑƏ',
 'Ə_ƐƐƐ__ƐƐƐ__ƓƓ__ƑƑƑƑ__ƓƓ__ƐƐƐ__ƔƔƔƔ__ƔƔƔƔ__ƒƒƒƒ__ƒƒƒƒ__ƑƑƑƑƏ',
 'Ə_ƒƒƒƒ__ƔƔƔƔ__ƔƔƔƔ__ƔƔƔƔ__ƑƑƑƑ__ƓƓ__ƓƓ__ƔƔƔƔ__ƓƓ__ƓƓ__ƓƓ__ƒƏ',
 'Ə_ƐƐƐ__ƔƔƔƔ__ƓƓ__ƒƒƒƒ__ƔƔƔƔ__ƓƓ__ƒƒƒƒ__ƑƑƑƑ__ƐƐƐ__ƔƔƔƔ__ƓƓ_Ə']

In [ ]:
from gtts import gTTS
from pydub import AudioSegment


# example text to speech
#myobj = gTTS(text='slip n go', lang="en", slow=False)
#myobj.save("welcome.mp3")
#os.system("welcome.mp3")

In [ ]:
sound = AudioSegment.from_mp3("sfx/template.mp3")

In [ ]:

def add_sound(self, sound):
    previous_sound = AudioSegment.from_mp3(self.output_fname)
    (previous_sound + sound).export(self.output_fname, format="mp3")

get_combination_by_id = lambda id_: [comb for comb in combinations if comb["id"] == id_]
def create_mp3(self):
    flag = False
    for idx, second in enumerate(self.workout):
        if second == EMPTY_PLACEHOLDER:
            flag = False
            continue
        if flag: continue

        comb_id = ord(second) - ASCII_OFFSET

        if comb_id == 0:
            # adding bell sound after each round
            self.add_sound(AudioSegment.from_mp3("bell.mp3"))
            continue

        combination = self.get_combination_by_id(comb_id)
        text = combination["name"]

        fname = "tmp"+str(comb_id)+".mp3"
        Speech(text, "en").save(fname)

        speech_sound = AudioSegment.from_mp3(fname) + 10
        silent_gap = AudioSegment.silent(self.comb_gap * 1000)

        latest_sound = silent_gap + speech_sound
        latest_sound += AudioSegment.silent((combination["time"]*1000 - len(speech_sound)))
        self.add_sound(latest_sound)

        flag = True

In [ ]:
gaps = []

cnt = 0
for char in workout.workout:
    if cnt != 0 and char == EMPTY_PLACEHOLDER:
        cnt += 1
    elif char == EMPTY_PLACEHOLDER and cnt == 0:
        cnt = 1
    else:
        gaps.append(cnt)
        cnt = 0

In [ ]:
import numpy as np

In [ ]:
np.unique(gaps)

In [ ]:
from pydub import AudioSegment
from google_speech import Speech


sound = AudioSegment.from_mp3("s.mp3")
print(len(sound))

sound = sound[1215000:]

print("exporting")
sound.export("cut.mp3", format="mp3")
